In [ ]:
#%pip install opentelemetry-sdk azure-core-tracing-opentelemetry
#%pip install ipynbname

In [1]:
import os, time
from azure.core.settings import settings

settings.tracing_implementation = "opentelemetry"
# Install opentelemetry with command "pip install opentelemetry-sdk".
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from azure.ai.agents.telemetry import AIAgentsInstrumentor

import dotenv
# Load environment variables from .env file
dotenv.load_dotenv()

True

In [2]:
# Setup tracing to console
# Requires opentelemetry-sdk
span_exporter = ConsoleSpanExporter()
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter))
trace.set_tracer_provider(tracer_provider)
tracer = trace.get_tracer(__name__)

AIAgentsInstrumentor().instrument()



In [4]:
# [START enable_tracing]
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor

# Enable Azure Monitor tracing
application_insights_connection_string = os.environ["APPLICATIONINSIGHTS_CONNECTION_STRING"]
configure_azure_monitor(connection_string=application_insights_connection_string)

scenario = "AI-AGENT_TRACING_MONITORING.ipynb"  # or any descriptive name
tracer = trace.get_tracer(__name__)

In [5]:
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

In [6]:
scenario = "AI-AGENT_TRACING_MONITORING.ipynb"
with tracer.start_as_current_span(scenario):
    with agents_client:
        agent = agents_client.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"], name="AQ-agent", instructions="You are helpful agent"
        )
        print(f"Created agent, agent ID: {agent.id}")

        thread = agents_client.threads.create()
        print(f"Created thread, thread ID: {thread.id}")

        message = agents_client.messages.create(thread_id=thread.id, role="user", content="Hello, what is the foundation of Azure AI?")
        print(f"Created message, message ID: {message.id}")

        run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
        print(f"Run completed with status: {run.status}")

        agents_client.delete_agent(agent.id)
        print("Deleted agent")

        messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        for msg in messages:
            if msg.text_messages:
                last_text = msg.text_messages[-1]
                print(f"{msg.role}: {last_text.text.value}")

Created agent, agent ID: asst_i8uNashal3H0DWjxNOgMsrFt
Created thread, thread ID: thread_XdKPmFQCd42dBH5rkAV1OpoA
Created message, message ID: msg_WCLwrPcbF3jJtx4gjHCalEEJ
Run completed with status: RunStatus.COMPLETED
Deleted agent
MessageRole.USER: Hello, what is the foundation of Azure AI?
MessageRole.AGENT: The foundation of **Azure AI** consists of a suite of artificial intelligence (AI) services, tools, and infrastructure provided by **Microsoft Azure** to help developers and organizations build, deploy, and manage AI solutions at scale. The foundation rests on several key pillars:

### 1. **Azure AI Infrastructure**
- **High-performance computing:** Access to powerful virtual machines, GPUs, and AI-optimized hardware for training and inference.
- **Scalability & Security:** Enterprise-grade cloud security, compliance, and scalability to handle any size workload.

### 2. **Foundation Models** (Large Language Models/LLMs & Multi-modal Models)
- **OpenAI models:** Integration of Op